In [1]:
#cd to the script folder
from google.colab import drive
import os
import sys
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/My Drive/Hackathon/DTM') #Need to change the address to where your script is
os.listdir('.')
#Please have the TEM images in dm4 or dm3 formats stored in 'test_data'
#Please have the U-Net model 'model.keras' here

Mounted at /content/drive


['DTMicroscope', 'test_data', 'model.keras', 'DTM_test.ipynb']

In [ ]:
!git clone https://github.com/pycroscopy/DTMicroscope.git

Cloning into 'DTMicroscope'...
remote: Enumerating objects: 921, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 921 (delta 23), reused 39 (delta 11), pack-reused 826 (from 1)
Receiving objects: 100% (921/921), 239.17 MiB | 17.13 MiB/s, done.
Resolving deltas: 100% (522/522), done.
Updating files: 100% (87/87), done.


In [2]:
'''There is currently a stupid mistake in the codes below for DTM installation'''
#ERROR: Could not find a version that satisfies the requirement DTMicroscope (from versions: none)
#ERROR: No matching distribution found for DTMicroscope
'''I'm trying to fix it'''

sys.path.insert(0, '/content/drive/My Drive/Hackathon/DTM/DTMicroscope')
!pip install DTMicroscope
!pip install SciFiReaders
!pip install pyNSID

ERROR: Could not find a version that satisfies the requirement DTMicroscope (from versions: none)
ERROR: No matching distribution found for DTMicroscope


In [11]:
import SciFiReaders
import pyNSID
from DTMicroscope.base import BaseMicroscope
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import time
import cv2
from IPython.display import display, clear_output
from tensorflow.keras.models import load_model

class TEM(BaseMicroscope):
  """I rewrote a new BaseMicroscope class for TEM data"""
  def __init__(self, data_path = None) -> None:
    super().__init__(data_path=data_path)
    #self._load_dataset_ram()

  def _sort_datasets(self):
      self.data_dict = glob(self.data_path + '*.dm4')

  def _load_dataset_ram(self):
      self.dataset = []
      for file in self.data_dict:
        reader = SciFiReaders.DM3Reader(file)
        self.dataset.append(reader.read())
      return

def split_image(image):
    """
    Splits a (2048, 2048) numpy array into 16 (512, 512) sub-images.

    Parameters:
        image (numpy.ndarray): Input image of shape (2048, 2048), dtype=float.

    Returns:
        numpy.ndarray: Array of shape (16, 512, 512), dtype=float, containing the sub-images.
    """
    if image.shape != (2048, 2048):
        raise ValueError("Input image must have shape (2048, 2048).")
    image = image.astype(np.float32)
    lb = np.percentile(image,0.8)
    ub = np.percentile(image,99.2)
    image = (image - lb) / (ub - lb)
    image[image < 0] = 0
    image[image > 1] = 1
    image = cv2.GaussianBlur(image, ksize=(0, 0), sigmaX=2, sigmaY=2)
    # Initialize output array to store 16 (512, 512) images
    sub_images = np.zeros((16, 512, 512), dtype=image.dtype)

    # Split the image into 512x512 chunks
    idx = 0
    for i in range(0, 2048, 512):
        for j in range(0, 2048, 512):
            sub_images[idx] = image[i:i+512, j:j+512]
            idx += 1
    return sub_images

def stitch_images(sub_images):
  image = np.zeros((2048,2048))
  idx = 0
  for i in range(0, 2048, 512):
    for j in range(0, 2048, 512):
      image[i:i+512, j:j+512] = np.argmax(sub_images[idx],axis = 2)
      idx += 1
  return image

In [4]:
model = load_model('model.keras')

In [20]:
M = TEM("test_data/")
M._load_dataset_ram()

t0 = time.time()
for itr in range(5000):
  '''this for loop mimics the continous viewing mode on TEM'''

  data = M.dataset[np.mod(itr,len(M.dataset))]
  t = time.time()
  data_np = data['Channel_000'].compute()
  data_np = cv2.resize(data_np,(2048,2048))

  '''make predictions'''
  data_sub = split_image(data_np)
  predictions = model.predict(data_sub, batch_size=1, verbose=0)
  data_pred = stitch_images(predictions)

  '''do some statistic'''
  fake_stats, bin = np.histogram(np.random.normal(loc=0, scale=1, size=1000), bins=100)
  bin = (bin[:-1]+bin[1:])/2

  '''plotting'''
  f,axs = plt.subplots(2,2, figsize = (10,10))
  axs[0][0].imshow(data_np,cmap = 'gray')
  axs[0][0].set_title('Original')
  axs[0][0].text(1580,100,'FPS: ' + '{:.2f}'.format(1/(t-t0)))

  axs[0][1].imshow(data_pred,cmap = 'gray')
  axs[0][1].set_title('U-Net prediction')

  axs[1][0].imshow(data_np,cmap = 'gray')
  axs[1][0].plot([200,1800,1000,200],[1800,1800,60,1800], linewidth = 2)
  axs[1][0].set_title('Contours (fake)')

  axs[1][1].bar(bin, fake_stats)
  axs[1][1].set_title('Statistics (fake)')
  t0 = time.time()
  plt.show()
  time.sleep(0.1)
  clear_output(wait=True)

KeyboardInterrupt: 